In [1]:
from pyspark.sql import SparkSession
import os

In [2]:
spark = SparkSession.builder \
    .appName("PSI Weather Analysis") \
    .master("yarn") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/27 06:43:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
psi_path = "hdfs:///data/psi_data.csv" 
weather_path = "hdfs:///data/weather_data.csv"

In [5]:
# read csv using spark

df_psi = spark.read.csv(
    psi_path,
    header=True,         # first row contains column names
    inferSchema=True     # attempts to infer column data types
)

df_weather = spark.read.csv(
    weather_path,
    header=True,
    inferSchema=True
)

print("PSI Data Row Count:", df_psi.count(), "rows")
df_psi.show(5)
print("Weather Data Row Count:", df_weather.count(), "rows")
df_weather.show(5)

PSI Data Row Count: 85488 rows
+-------------+-----+-----+----+----+-------+
|     24hr_psi|north|south|east|west|central|
+-------------+-----+-----+----+----+-------+
|1/4/2014 1:00|   55|   54|  54|  58|     54|
|1/4/2014 2:00|   55|   54|  54|  59|     54|
|1/4/2014 3:00|   55|   55|  54|  60|     55|
|1/4/2014 4:00|   56|   55|  55|  62|     55|
|1/4/2014 5:00|   57|   55|  56|  63|     56|
+-------------+-----+-----+----+----+-------+
only showing top 5 rows

Weather Data Row Count: 46248 rows
+---------+-------------------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+----------------+-------------------+--------------------+
|     name|           datetime|temp|feelslike| dew|humidity|precip|precipprob|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|solarradiation|solarenergy|uvindex|severerisk|    

# Convert 24hr_psi to timestamp column

In [ ]:
# convert 24hr_psi to timestamp column

from pyspark.sql import functions as F

# Adjust the format string depending on your actual data format:
# e.g., "M/d/yyyy H:mm" if months/days/hours are not zero-padded.
# If your data is like "01/01/2023 00:00" (fully zero-padded), use "MM/dd/yyyy HH:mm".
df_psi = df_psi.withColumn(
    "parsed_datetime",
    F.to_timestamp("24hr_psi", "d/M/yyyy H:mm")  # or "MM/dd/yyyy HH:mm"
)
print("Row Count:", df_psi.count(), "rows")


Row Count: 85488 rows


In [8]:
# filter to keep 2022 and 2023

df_psi_filtered = df_psi.filter(
    (F.year("parsed_datetime") >= 2022) &
    (F.year("parsed_datetime") <= 2023)
)

print("After filtering PSI data (2022-2023):", df_psi_filtered.count(), "rows")
df_psi_filtered.show(5)

After filtering PSI data (2022-2023): 17520 rows
+-------------+-----+-----+----+----+-------+-------------------+
|     24hr_psi|north|south|east|west|central|    parsed_datetime|
+-------------+-----+-----+----+----+-------+-------------------+
|1/1/2022 0:00|   29|   27|  32|  17|     29|2022-01-01 00:00:00|
|1/1/2022 1:00|   30|   25|  32|  18|     29|2022-01-01 01:00:00|
|1/1/2022 2:00|   30|   24|  31|  19|     30|2022-01-01 02:00:00|
|1/1/2022 3:00|   30|   23|  30|  19|     31|2022-01-01 03:00:00|
|1/1/2022 4:00|   31|   23|  29|  20|     30|2022-01-01 04:00:00|
+-------------+-----+-----+----+----+-------+-------------------+
only showing top 5 rows



# Convert 24hr_weather string to timestamp column

In [9]:
# Convert the string column (e.g., "datetime") to a proper timestamp.
# Adjust the format string to match your actual data (for example, "yyyy-MM-dd'T'HH:mm:ss").
df_weather = df_weather.withColumn(
    "parsed_datetime",
    F.to_timestamp("datetime", "yyyy-MM-dd'T'HH:mm:ss")
)
print("Row Count:", df_weather.count(), "rows")


Row Count: 46248 rows


In [12]:
# Filter for rows with year 2022 or 2023.
df_weather_filtered = df_weather.filter(
    (F.year("parsed_datetime") >= 2022) &
    (F.year("parsed_datetime") <= 2023)
)

print("After filtering Weather data (2022-2023):", df_weather_filtered.count(), "rows")
df_weather_filtered.show(5)

After filtering Weather data (2022-2023): 18504 rows
+---------+-------------------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+--------------------+-------------------+--------------------+-------------------+
|     name|           datetime|temp|feelslike| dew|humidity|precip|precipprob|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|solarradiation|solarenergy|uvindex|severerisk|          conditions|               icon|            stations|    parsed_datetime|
+---------+-------------------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+--------------------+-------------------+--------------------+-------------------+
|Singapore|2022-01-01 00:00:00|25.0|     25.0|24.0

# Split Date/Time & Clean Data

In [ ]:
# for psi, split into date/hour

df_psi_filtered = (
    df_psi_filtered
    .withColumn("psi_date", F.to_date("parsed_datetime"))
    .withColumn("psi_hour", F.hour("parsed_datetime"))
    .drop("24hr_psi")         # The original string date-time column
    .drop("parsed_datetime")  # If not needed anymore
)
print("Row Count:", df_psi_filtered.count(), "rows")
df_psi_filtered.show(5)



Row Count: 17520 rows
+-----+-----+----+----+-------+----------+--------+
|north|south|east|west|central|  psi_date|psi_hour|
+-----+-----+----+----+-------+----------+--------+
|   29|   27|  32|  17|     29|2022-01-01|       0|
|   30|   25|  32|  18|     29|2022-01-01|       1|
|   30|   24|  31|  19|     30|2022-01-01|       2|
|   30|   23|  30|  19|     31|2022-01-01|       3|
|   31|   23|  29|  20|     30|2022-01-01|       4|
+-----+-----+----+----+-------+----------+--------+
only showing top 5 rows



In [ ]:
# for weather, split into date/hour

df_weather_filtered = (
    df_weather_filtered
    .withColumn("weather_date", F.to_date("parsed_datetime"))
    .withColumn("weather_hour", F.hour("parsed_datetime"))
    .drop("datetime")
    .drop("parsed_datetime")
)
print("Row Count:", df_weather_filtered.count(), "rows")
df_weather_filtered.show(5)



Row Count: 18504 rows
+---------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+--------------------+-------------------+--------------------+------------+------------+
|     name|temp|feelslike| dew|humidity|precip|precipprob|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|solarradiation|solarenergy|uvindex|severerisk|          conditions|               icon|            stations|weather_date|weather_hour|
+---------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+--------------------+-------------------+--------------------+------------+------------+
|Singapore|25.0|     25.0|24.0|   94.03|   0.0|         0|      NULL|   0|        0|    25.9|      5.2|   32.0|          10

# Merge on Common Keys

In [15]:
df_filtered_merged = df_psi_filtered.join(
    df_weather_filtered,
    (df_psi_filtered.psi_date == df_weather_filtered.weather_date) &
    (df_psi_filtered.psi_hour == df_weather_filtered.weather_hour),
    how="inner"  # or "left", "right", etc.
)
print("Row Count:", df_filtered_merged.count(), "rows")
df_filtered_merged.show(5)
df_filtered_merged.printSchema()


Row Count: 18504 rows


25/07/27 06:55:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+-----+----+----+-------+----------+--------+---------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+--------------------+-------------------+--------------------+------------+------------+
|north|south|east|west|central|  psi_date|psi_hour|     name|temp|feelslike| dew|humidity|precip|precipprob|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|solarradiation|solarenergy|uvindex|severerisk|          conditions|               icon|            stations|weather_date|weather_hour|
+-----+-----+----+----+-------+----------+--------+---------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+--------------------+-------------------+--------------------+------------+---------

# Save Merged Data to HDFS

In [16]:
hdfs_output_path = "hdfs:///data/psi_weather_merged.parquet"

df_filtered_merged.write.mode("overwrite").parquet(hdfs_output_path)


In [17]:
# verify by listing directory
# Read the Parquet file from HDFS
df_hdfs = spark.read.parquet("hdfs:///data/psi_weather_merged.parquet")

print("Row Count:", df_filtered_merged.count(), "rows")

# Show the first few rows
df_hdfs.show(5)



Row Count: 18504 rows
+-----+-----+----+----+-------+----------+--------+---------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+--------------------+-------------------+--------------------+------------+------------+
|north|south|east|west|central|  psi_date|psi_hour|     name|temp|feelslike| dew|humidity|precip|precipprob|preciptype|snow|snowdepth|windgust|windspeed|winddir|sealevelpressure|cloudcover|visibility|solarradiation|solarenergy|uvindex|severerisk|          conditions|               icon|            stations|weather_date|weather_hour|
+-----+-----+----+----+-------+----------+--------+---------+----+---------+----+--------+------+----------+----------+----+---------+--------+---------+-------+----------------+----------+----------+--------------+-----------+-------+----------+--------------------+-------------------+--------------------+